In [2]:
import sqlite3
from sqlite3 import Error
import pandas as pd
from google.colab import drive
import pandas as pd 
import numpy as np

drive.mount('/content/gdrive')
# !ls "/content/gdrive/My Drive"
data_path = "/content/gdrive/My Drive/Master ADS/"

Mounted at /content/gdrive


In [3]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn

def run_query(conn, query):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    # Create a cursor
    cur = conn.cursor()    
    # Send the query to the database
    cur.execute(query)
    # Extract the results of the query
    results = cur.fetchall()
    # Return the results
    return results

# Question 1
Write python code that uses python connectors to extract all the tables from the database and save them to .csv files. 


In [4]:
database = data_path + "chinook.db"
conn = create_connection(database) 
cursor = conn.cursor()
tables = cursor.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()

for table in tables:
  if 'sqlite' in table[0]:
    pass 
  else:
    cursor.execute("SELECT * FROM " + table[0])
    headers = [i[0] for i in cursor.description]
    rows = cursor.fetchall()
    df = pd.DataFrame(rows)
    df.columns = headers
    df.to_csv(data_path + table[0] + ".csv", encoding='utf-8')

# Question 2
Write the queries that you proposed in Task2 using python. You will need to read the tables that will be used in the queries into Pandas Dataframes and write the queries to extract the data from the Dataframes directly (do not use SQL on the original database).

In [18]:
from unicodedata import name
# reading the tables we need for the queries
artists_table = pd.read_csv(data_path + "artists.csv")
tracks_table = pd.read_csv(data_path + "tracks.csv")
albums_table = pd.read_csv(data_path + "albums.csv")
employees_table = pd.read_csv(data_path + "employees.csv")

# query 1
merged = pd.merge(artists_table, albums_table, on='ArtistId', how='left').sort_values('Name')
query1 = merged.filter(['Name','AlbumId'], axis=1)
print("Result of query 1: {}".format(query1))
print("Rows returned: {}\n".format(len(query1)))

# query 2
query2 = tracks_table['TrackId'][tracks_table['Composer'].isna()]
print("Result of query 2: {}".format(query2))
print("Rows returned: {}\n".format(len(query2)))

# query 3
query3 = tracks_table['Name'][tracks_table['UnitPrice'] != 0.99]
print("Result of query 3: {}".format(query3))
print("Rows returned: {}\n".format(len(query3)))

# query 4
aux_4 = tracks_table.loc[tracks_table['Milliseconds'] < 120000]
query4 = aux_4['Name']
print("Result of query 4: {}".format(query4))
print("Rows returned: {}\n".format(len(query4)))

# query 5
query5 = tracks_table.groupby(["MediaTypeId"])["MediaTypeId"].count()
print("Result of query 5: {}".format(query5))
print("Rows returned: {}\n".format(len(query5)))

# query 6
query6 = tracks_table.agg(['min', 'max'])['Bytes']
print("Result of query 6: {}".format(query6))
print("Rows returned: {}\n".format(len(query6)))

# query 7
query7 = pd.merge(employees_table, employees_table, left_on='ReportsTo', right_on='EmployeeId', how='left')\
           .filter(['FirstName_y','LastName_y', 'FirstName_x','LastName_x'])\
           .sort_values(by='FirstName_x')\
           .dropna()

print("Result of query 7: {}".format(query7))
print("Rows returned: {}\n".format(len(query7)))    

Result of query 1:                                           Name  AlbumId
71                                A Cor Do Som      NaN
0                                        AC/DC      1.0
1                                        AC/DC      4.0
368  Aaron Copland & London Symphony Orchestra    296.0
337                             Aaron Goldberg    267.0
..                                         ...      ...
316                                        Xis      NaN
397                             Yehudi Menuhin    325.0
348                                   Yo-Yo Ma    278.0
303                             Youssou N'Dour      NaN
288                             Zeca Pagodinho    248.0

[418 rows x 2 columns]
Rows returned: 418

Result of query 2: 1          2
62        63
63        64
64        65
65        66
        ... 
3477    3478
3480    3481
3495    3496
3496    3497
3498    3499
Name: TrackId, Length: 978, dtype: int64
Rows returned: 978

Result of query 3: 2818    Battlestar Gala

In [9]:
# querys from Task2

# show all artists and the albums that belong to them
query1 = "SELECT artists.Name, albums.AlbumId\
          FROM artists\
          LEFT JOIN albums ON artists.ArtistId = albums.ArtistId\
          ORDER BY artists.Name;"

# show id of the tracks that have no composer information
query2 = "SELECT TrackId\
          FROM tracks\
          WHERE Composer IS NULL;"

# show names of the tracks where unit price is not 0.99
query3 = "SELECT Name\
          FROM tracks\
          WHERE NOT UnitPrice == 0.99;"


# show all tracks that have a length of less than 120000 milliseconds (2minutes).
query4 = "SELECT Name\
          FROM tracks\
          WHERE Milliseconds < 120000;"

# show the number of records of the tracks per mediatypeId
query5 = "SELECT MediaTypeId, COUNT(MediaTypeId) as 'Sum'\
          From tracks\
          GROUP BY MediaTypeId;"

# show the tracks with the highest storage value and the lowest one
query6 = "SELECT MIN(Bytes) AS LowestStorage, MAX(Bytes) AS HightestStorage\
          FROM tracks;"

# matches the tracks that last the same amount of time
query7 = "SELECT m.firstname, m.lastname, e.firstname, e.lastname\
          FROM employees e\
          INNER JOIN employees m ON m.employeeid = e.reportsto\
          ORDER BY m.firstname;"

# query_list = [query1, query2, query3, query4, query5, query6, query7]
query_list = [query1, query2, query3, query4, query5, query6, query7]

# create a database connection   
with conn:
  for i, query in enumerate(query_list):
    data = run_query(conn, query)
    print("Result of the query {}: {}".format(i+1, data))
    print("Rows returned: {}".format(len(data)))

Result of the query 1: [('A Cor Do Som', None), ('AC/DC', 1), ('AC/DC', 4), ('Aaron Copland & London Symphony Orchestra', 296), ('Aaron Goldberg', 267), ('Academy of St. Martin in the Fields & Sir Neville Marriner', 280), ('Academy of St. Martin in the Fields Chamber Ensemble & Sir Neville Marriner', 281), ('Academy of St. Martin in the Fields, John Birch, Sir Neville Marriner & Sylvia McNair', 288), ('Academy of St. Martin in the Fields, Sir Neville Marriner & Thurston Dart', 327), ('Academy of St. Martin in the Fields, Sir Neville Marriner & William Bennett', None), ('Accept', 2), ('Accept', 3), ('Adrian Leaper & Doreen de Feis', 330), ('Aerosmith', 5), ("Aerosmith & Sierra Leone's Refugee Allstars", None), ('Aisha Duo', 262), ('Alanis Morissette', 6), ('Alberto Turco & Nova Schola Gregoriana', 272), ('Alice In Chains', 7), ('Amy Winehouse', 321), ('Amy Winehouse', 322), ('Anne-Sophie Mutter, Herbert Von Karajan & Wiener Philharmoniker', 275), ('Antal Doráti & London Symphony Orchest